In [2]:
import numpy as np
import pandas as pd

In [32]:
class timetable:
    cost_multiplier = np.array([0,0,9,9,9,18,18,36,36,235])
    cost_additive = np.array([0,50,50,100,200,200,300,300,400,500])

    cost_multiplier_matrix = np.resize(cost_multiplier, (100,10))
    cost_additive_matrix = np.resize(cost_additive, (100, 10))

    def __init__(self):
        self.timetable = np.zeros((100, 5000, 10))
        
    def assign_family_by_choice(self, fam_id, choice):
        self.timetable[:,fam_id,:] = np.zeros((100,10))
        day = df.iloc[fam_id, choice + 1]
        self.timetable[day + 1, fam_id, choice] = df.loc[fam_id, 'n_people']
        
    def assign_family_by_day(self, fam_id, day):
        self.timetable[:,fam_id,:] = np.zeros((100,10))
        choice = np.where(df.iloc[fam_id] == day)[0][0]
        self.timetable[day + 1, fam_id, choice] = df.loc[fam_id, 'n_people']
        
    def get_preference_cost(self):
        multiple_cost = np.sum(self.timetable, axis=1) * self.cost_multiplier_matrix
        additive_cost = np.count_nonzero(self.timetable, axis=1) * self.cost_additive_matrix
    
        return np.sum(multiple_cost + additive_cost)

In [33]:
df = pd.read_csv('family_data.csv')
df.head()

,family_id,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
0,0,52,38,12,82,33,75,64,76,10,28,4
1,1,26,4,82,5,11,47,38,6,66,61,4
2,2,100,54,25,12,27,82,10,89,80,33,3
3,3,2,95,1,96,32,6,40,31,9,59,2
4,4,53,1,47,93,26,3,46,16,42,39,4


In [34]:
# Create timetable object
schedule = timetable()

In [38]:
# assign a family one of their choices
schedule.assign_family_by_choice(fam_id=0, choice=3)
print(schedule.timetable[83,0])

[0. 0. 0. 4. 0. 0. 0. 0. 0. 0.]


In [39]:
# calculate the preference cost
schedule.get_preference_cost()

136.0

In [30]:
schedule.timetable[:,0,:] = np.zeros((100,10))

In [31]:
schedule.timetable[:,0,:]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 